<a href="https://colab.research.google.com/github/ailunguo/Test/blob/main/Tensorflow_Test/%E5%B1%82API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 层API

# 基础层类

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
layer = layers.Dense(32, activation='relu')
inputs = tf.random.uniform(shape=(10, 20))
outputs = layer(inputs)

In [ ]:
# 层维护一个状态，当层在训练期间接受数据时更新，并存储在layer.weights
layer.weights

[<tf.Variable 'dense/kernel:0' shape=(20, 32) dtype=float32, numpy=
 array([[-0.05567047,  0.31292653, -0.13742629, -0.2273688 ,  0.26214993,
         -0.23777099, -0.04089954, -0.20524241,  0.30012488,  0.32383823,
          0.30036312, -0.02244112,  0.08285853,  0.18708074,  0.1158919 ,
         -0.2807782 , -0.2572527 , -0.19771299,  0.14625481,  0.30086982,
          0.08598122,  0.03751105, -0.13985816,  0.0642311 , -0.1418638 ,
         -0.33267656, -0.03955612,  0.3238871 , -0.03714079,  0.17921627,
         -0.04312432, -0.27468476],
        [ 0.32543623,  0.239528  , -0.00789338,  0.01627186,  0.26870483,
         -0.0443435 ,  0.29473317, -0.08163288, -0.32503843, -0.25181764,
          0.10047501,  0.19757944, -0.24360624,  0.10487357, -0.3163072 ,
          0.20873171,  0.28273028, -0.04418761,  0.0934256 , -0.26690453,
          0.19274998,  0.24786443,  0.3316276 ,  0.19501501,  0.08874011,
          0.16809136, -0.28583744, -0.25757354, -0.14694855, -0.0962991 ,
        

## 基础层类

In [ ]:
# Layer类
tf.keras.layers.Layer(
    trainable==True, name=None, dtype=None, dynamic=False, **kwargs
)
# dynamic：布尔值，False则生成静态计算图，True就是要急切的运行

In [ ]:
# 实例
class SimpleDense(layers.Layer):

  def __init__(self, units=32):
    super(SimpleDense, self).__init__()
    self.units = units

  def build(self, input_shape):
    w_init = tf.random_normal_initializer()
    self.w = tf.Variable(
        initial_value=w_init(shape=(input_shape[-1], self.units),
                     dtype='float32'),
        trainable=True)
    b_init = tf.zeros_initializer()
    self.b = tf.Variable(
        initial_value=b_init(shape=(self.units,), dtype='float32'),
        trainable=False)

  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b

linear_layer = SimpleDense(4)

y = linear_layer(tf.ones((2, 2)))
assert len(linear_layer.weights) == 2

assert len(linear_layer.trainable_weights) == 1

In [ ]:
tf.random_normal_initializer()(shape=(2, 2), dtype='float32')

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-0.00342907, -0.00129643],
       [ 0.0324378 ,  0.05942843]], dtype=float32)>

In [ ]:
# add_weight()方法提供了创建权重的方法
# 改进上述SimpleDense
class SimpleDense(layers.Layer):

  def __init__(self, units=32):
    super(SimpleDense, self).__init__()
    self.units = units

  def build(self, input_shape):
    self.w = self.add_weight(shape=(input_shape[-1], self.units),
                  initializer='random_normal',
                  trainable=True)
    self.b = self.add_weight(shape=(self.units,),
                  initializer='ramdom_normal',
                  trainable=True)

  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b


In [ ]:
# 计算输入的运行总和的实例层
class ComputeSum(layers.Layer):

  def __init__(self, input_dim):
    super(ComputeSum, self).__init__()
    self.total = tf.Variable(initial_value=tf.zeros((input_dim,)),
                  trainable=False)

  def call(self, inputs):
    self.total.assign_add(tf.reduce_sum(inputs, axis=0))
    return self.total

In [ ]:
tf.reduce_sum(tf.ones((2,2)), axis=0)

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([2., 2.], dtype=float32)>

In [ ]:
my_sum = ComputeSum(2)
x = tf.ones((2, 2))

y = my_sum(x)
print(y.numpy())

y = my_sum(x) # 继续累加，在权重上的累加
print(y.numpy())
print(my_sum.weights)
print(my_sum.total)
assert my_sum.weights == [my_sum.total]
assert my_sum.non_trainable_weights == [my_sum.total]
assert my_sum.trainable_weights == []

[2. 2.]
[4. 4.]
[<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([4., 4.], dtype=float32)>]
<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([4., 4.], dtype=float32)>


add_weight方法

In [ ]:
from keras.src.layers.rnn.base_conv_lstm import constraints
from keras.api._v2.keras import regularizers
layers.Layer.add_weight(
    name=None,
    shape=None,
    dtype=None,
    initializer=None,
    regularizer=None, # 正则化器实例
    trainable=None,
    constraint=None,
    use_resource=None,
    synchronization=tf.VariableSynchronization.AUTO,
    aggregation=tf.VariableSynchronization.NONE,
    **kwargs
)


In [ ]:
# 层的权重代表该层的状态，其也可以加载到类似的参数化层中
layer_a = tf.keras.layers.Dense(1,
      kernel_initializer=tf.constant_initializer(1.))
a_out = layer_a(tf.convert_to_tensor([[1., 2., 3.]]))
layer_a.get_weights() # shape=3,1 因为输入为3，输出为1

[array([[1.],
        [1.],
        [1.]], dtype=float32),
 array([0.], dtype=float32)]

In [ ]:
layer_b = tf.keras.layers.Dense(1,
      kernel_initializer=tf.constant_initializer(2,))
b_out = layer_b(tf.convert_to_tensor([[10., 20., 30.]]))
layer_b.get_weights()

[array([[2.],
        [2.],
        [2.]], dtype=float32),
 array([0.], dtype=float32)]

In [ ]:
layer_b.set_weights(layer_a.get_weights())
layer_b.get_weights()

[array([[1.],
        [1.],
        [1.]], dtype=float32),
 array([0.], dtype=float32)]

In [ ]:
# get_config()方法,返回模型的配置
layer_b.get_config()

{'name': 'dense_1',
 'trainable': True,
 'dtype': 'float32',
 'units': 1,
 'activation': 'linear',
 'use_bias': True,
 'kernel_initializer': {'module': 'tensorflow.python.ops.init_ops_v2',
  'class_name': 'Constant',
  'config': {'value': 2},
  'registered_name': 'Constant'},
 'bias_initializer': {'module': 'keras.initializers',
  'class_name': 'Zeros',
  'config': {},
  'registered_name': None},
 'kernel_regularizer': None,
 'bias_regularizer': None,
 'activity_regularizer': None,
 'kernel_constraint': None,
 'bias_constraint': None}

In [ ]:
# add_loss方法，添加损失张量
# Layer.add_loss(losses, **kwargs)
# 例子
class MyLayer(tf.keras.layers.Layer):
  def call(self, inputs):
    self.add_loss(tf.abs(tf.reduce_mean(inputs)))
    return inputs

In [ ]:
# add_loss也可以在构建模型的时候直接加到功能模型上调用
# 并且这将在模型的拓扑中被追踪
# 例子
inputs = tf.keras.Input(shape=(10,))
x = tf.keras.layers.Dense(10)(inputs)
outputs = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs, outputs)

model.add_loss(tf.abs(tf.reduce_mean(x)))

In [ ]:
# 将损失包装在零参数lambda中，不直接参与训练
inputs = tf.keras.Input(shape=(10,))
d = tf.keras.layers.Dense(10)
x = d(inputs)
outputs = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs, outputs)

model.add_loss(lambda: tf.reduce_mean(d.kernel))

In [ ]:
# losses 返回add_loss中的函数的值
# tf.keras.layers.Layer.losses
import numpy as np
class MyLayer(tf.keras.layers.Layer):
  def call(self, inputs):
    self.add_loss(tf.abs(tf.reduce_mean(inputs)))
    return inputs
l = MyLayer()
l(np.ones((10, 1)))
l.losses

[<tf.Tensor: shape=(), dtype=float32, numpy=1.0>]

In [ ]:
inputs = tf.keras.Input(shape=(10,))
x = tf.keras.layers.Dense(10)(inputs)
outputs = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs, outputs)
print(len(model.losses))
model.add_loss(tf.abs(tf.reduce_mean(x)))
model.add_loss(tf.reduce_sum(x))
print(len(model.losses))

0
2


In [ ]:
inputs = tf.keras.Input(shape=(10,))
d = tf.keras.layers.Dense(10, kernel_initializer='ones')
x = d(inputs)
outputs = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs, outputs)
model.add_loss(lambda: tf.reduce_mean(d.kernel))
model.losses

[<tf.Tensor: shape=(), dtype=float32, numpy=1.0>]

# 层激活函数

In [1]:
# model.add(layers.Dense(64, activation=activations.relu))

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import activations

model.add(layers.Dense(64))
model.add(layers.Activation(activations.relu))
# 相当于model.add(layers.Dense(64, activation=activations.relu))

In [ ]:
# 所有内置激活也可以通过字符串标识符传递
model.add(layers.Dense(64, activation='relu'))

In [3]:
# 激活函数
# relu的功能
# tf.keras.activations.relu(x, alpha=0.0, max_value=None, threshold=0.0)
# 上述relu函数就是这个max(x, 0)，0是阙值,可以改变


In [4]:
import tensorflow as tf

In [5]:
# 例子
foo = tf.constant([-10, -5, 0.0, 5, 10], dtype=tf.float32)
print("default:", tf.keras.activations.relu(foo).numpy())
print("alpha=0.5:",tf.keras.activations.relu(foo, alpha=0.5).numpy)
print("max_value=5.:", tf.keras.activations.relu(foo, max_value=5.).numpy())
print("threshold=5.:", tf.keras.activations.relu(foo, threshold=5.))
# alpha:控制低于阙值的值的斜率
# max_value: 设置饱和阙值
# threshold: 阙值

default: [ 0.  0.  0.  5. 10.]
alpha=0.5: <bound method _EagerTensorBase.numpy of <tf.Tensor: shape=(5,), dtype=float32, numpy=array([-5. , -2.5,  0. ,  5. , 10. ], dtype=float32)>>
max_value=5.: [0. 0. 0. 5. 5.]
threshold=5.: tf.Tensor([-0. -0.  0.  0. 10.], shape=(5,), dtype=float32)


In [7]:
# sigmoid函数
# tf.keras.activations.sigmoid(x)
# sigmoid(x) = 1 / (1 + exp(-x))

In [8]:
# 例子
a = tf.constant([-20, -1.0, 0.0, 1.0, 20], dtype=tf.float32)
b = tf.keras.activations.sigmoid(a)
b.numpy()

array([2.0611537e-09, 2.6894143e-01, 5.0000000e-01, 7.3105860e-01,
       1.0000000e+00], dtype=float32)

In [9]:
# softmax函数
# softmax(x) = exp(x) / tf.reduce_sum(exp(x))

# 单独使用
inputs = tf.random.normal(shape=(32, 10))
outputs = tf.keras.activations.softmax(inputs)
tf.reduce_sum(outputs[0, :])

<tf.Tensor: shape=(), dtype=float32, numpy=0.99999994>

# 层权重初始化器

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import initializers

layer = layers.Dense(
    units=64,
    kernel_initializer = initializers.RandomNormal(stddev=0.01),
    bias_initializer = initializers.Zeros()
)

layer = layers.Dense(
    units = 64,
    kernal_initializer='random_normal',
    bias_initializer = 'zeros'
)

RandomNormal初始化类

In [ ]:
# RandomNormal初始化类
tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)

In [ ]:
# 例子
initializer = tf.keras.initializers.RandomNormal(mean=0., stddev=1.)
values = initializer(shape=(2, 2))
print(values)

layer = tf.keras.layers.Dense(3, kernel_initializer=initializer)

RandomUniform类

In [ ]:
tf.keras.initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=None)

In [10]:
# 例子
initializer = tf.keras.initializers.RandomUniform(minval=0., maxval=1.)
values = initializer(shape=(2, 2))
print(values)
layer = tf.keras.layers.Dense(3, kernel_initializer=initializer)

tf.Tensor(
[[0.28100932 0.05351639]
 [0.48277593 0.05010784]], shape=(2, 2), dtype=float32)


Zeros类

In [ ]:
tf.keras.initializers.Zeros()

In [12]:
initializer = tf.keras.initializers.Zeros()
values = initializer(shape=(2, 2))
print(values)
layer = tf.keras.layers.Dense(3, kernel_initializer=initializer)

tf.Tensor(
[[0. 0.]
 [0. 0.]], shape=(2, 2), dtype=float32)


Ones类

In [ ]:
tf.keras.initializers.Ones()

In [13]:
initializer = tf.keras.initializers.Ones()
print(initializer(shape=(2, 2)))
layer = tf.keras.layers.Dense(3, kernel_initializer=initializer)

tf.Tensor(
[[1. 1.]
 [1. 1.]], shape=(2, 2), dtype=float32)


Constant类

In [ ]:
tf.keras.initializers.Constant(value=0)

In [14]:
# 例子
initializer = tf.keras.initializers.Constant(3.)
print(initializer(shape=(2, 2)))
layer = tf.keras.layers.Dense(3, kernel_initializer=initializer)

tf.Tensor(
[[3. 3.]
 [3. 3.]], shape=(2, 2), dtype=float32)


创建自定义初始值设定项

In [ ]:
def my_init(shape, dtype=None):
  return tf.random.normal(shape, dtype=dtype)

layer = Dense(3, kernel_initializer=my_init)

In [ ]:
# Initializer子类
class ExampleRandomNoraml(tf.keras.initializers.Initializer):

  def __init__(self, mean, stddev):
    self.mean = mean
    self.stddev = stddev

  def __call__(self, shape, dtype=None):
    return tf.random.normal(
        shape,
        mean = self.mean,
        stddev=self.stddev,
        dtype=dtype
    )

  def get_config(self):
    return {'mean': self.mean, 'stddev': self.stddev}

# 层权重正则化器

In [ ]:
# 正则化惩罚是按层应用的
# kernel_regularizer: 正则化器对层的内核应用惩罚
# bias_regularizer: 正则化器对层的偏差施加惩罚
# activity_regularizer: 正则化器对层的输出应用惩罚

In [16]:
from tensorflow.keras import layers
from tensorflow.keras import regularizers

layer = layers.Dense(
    units=64,
    kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
    bias_regularizer=regularizers.L2(1e-4),
    activity_regularizer=regularizers.L2(1e-5)
)

In [18]:
layer = layers.Dense(
    units=5,
    kernel_initializer='ones',
    kernel_regularizer=regularizers.L1(0.01),
    activity_regularizer=regularizers.L2(0.01)
)
tensor = tf.ones(shape=(5, 5)) * 2.0
out = layer(tensor)

print(tf.math.reduce_sum(layer.losses))

tf.Tensor(5.25, shape=(), dtype=float32)


In [20]:
tf.square(tf.ones(shape=(5, 5)) * 2.0)

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[4., 4., 4., 4., 4.],
       [4., 4., 4., 4., 4.],
       [4., 4., 4., 4., 4.],
       [4., 4., 4., 4., 4.],
       [4., 4., 4., 4., 4.]], dtype=float32)>

In [ ]:
dense = tf.keras.layers.Dense(3, kernel_regularizer='l1')
dense = tf.keras.layers.Dense(3, kernel_regularizer='l2')
dense = tf.keras.layers.Dense(3, kernel_regularizer='l1_l2')

In [21]:
# 创建自定义正则化器
def my_regularizer(x):
  return 1e-3 * tf.reduce_sum(tf.square(x))

In [22]:
# Regularizer子类
class MyRegularizer(regularizers.Regularizer):

  def __init__(self, strength):
    self.strength = strength

  def __call__(self, x):
    return self.strength * tf.reduce_sum(tf.square(x))

In [24]:
class my_class:
  def __init__(self, init_value):
    self.init_value = init_value

  def __call__(self, call_value):
    return self.init_value + call_value
my_cl = my_class(10)
print(my_cl(20))

30
